In [ ]:
from nhlpy.api.query.builder import QueryBuilder, QueryContext, QueryBase
from nhlpy.nhl_client import NHLClient
from nhlpy.api.query.filters.draft import DraftQuery
from nhlpy.api.query.filters.season import SeasonQuery
from nhlpy.api.query.filters.game_type import GameTypeQuery
from nhlpy.api.query.filters.position import PositionQuery, PositionTypes
import polars as pl

pl.Config.set_tbl_cols(50)
pl.Config.set_fmt_str_lengths(20)
pl.Config.set_tbl_width_chars(100)
pl.Config.set_tbl_rows(200)
pl.Config.set_tbl_formatting('NOTHING')
pl.Config.set_tbl_column_data_type_inline(True)  
pl.Config.set_tbl_dataframe_shape_below(True) 

In [ ]:
client = NHLClient(verbose=False)

In [ ]:
# Example player query
filters: list[QueryBase] = [
    GameTypeQuery(game_type="2"),
    # DraftQuery(year="2020", draft_round="2"),
    SeasonQuery(season_start="20242025", season_end="20242025"),
    PositionQuery(position=PositionTypes.ALL_FORWARDS),
]

query_builder = QueryBuilder()
query_context: QueryContext = query_builder.build(filters=filters)

data = client.stats.skater_stats_with_query_context(
    report_type='summary',
    query_context=query_context,
    aggregate=True
)

data = pl.DataFrame(data['data'])

crosby = data.filter(pl.col('skaterFullName') == 'Sidney Crosby')

ppg = crosby.select('pointsPerGame').item()
print(ppg)

In [ ]:
# Dealing with 'start' + 'limit' args is annoying here, so we just query by season
start_season: int = 2010
end_season: int = 2024
seasons: list[str] = [f'{season}{season+1}' for season in range(start_season, end_season + 1)]

In [ ]:
# Example aggregate team data -- not too much info here honestly
dfs = [ ]
for season in seasons:
  data = client.stats.team_summary(season, season, is_aggregate=False, limit=100) 
  dfs.append(pl.DataFrame(data))
team_stats: pl.DataFrame = pl.concat(dfs)

In [ ]:
# Example team standings -- this gives us some more information
dfs = [ ]
for season in seasons:
  # For some reason I have to use 'now' for 2024
  data = client.standings.get_standings(season=season)['standings'] if season != '20242025' else client.standings.get_standings(date='now')['standings']
  df = pl.DataFrame(data)
  dfs.append(df)
team_standings: pl.DataFrame = pl.concat(dfs, how='diagonal') # need diagonal since not all columns are available for every season -- may want to filter columns with missing data
team_standings


In [ ]:
# Get only playoff teams this year
standings = client.standings.get_standings(date='now')['standings']
playoff_teams: list[str] = [team['teamAbbrev']['default'] for team in standings if team['clinchIndicator'] != 'e']

In [ ]:
schedule = client.schedule.get_season_schedule('PIT', '20172018')['games']
playoff = [game for game in schedule if game['gameType'] == 3]
len(playoff)


In [ ]:
# Game data
season = '20242025'
standings = client.standings.get_standings(season=season)['standings'] if season != '20242025' else client.standings.get_standings(date='now')['standings']
teams: list[str] = [team['teamAbbrev']['default'] for team in standings]

# make sure we don't query the same game twice
seen_game_ids: set[int] = set()

for team in teams:
    print(season, team)
    games: list[dict] = client.schedule.get_season_schedule(team, season)['games']

    # Get information about a game
    games: list[dict] = [ ]
    for game in games:
        game_id: int = game['id']
        game_type: int = game['gameType']

        # only care about playoff and regular season
        if game_type != 2 and game_type != 3:
            continue

        if game_id in seen_game_ids:
            continue

        seen_game_ids.add(game_id)
        data = client.game_center.game_story(game_id)
        team_summary = data['summary']['teamGameStats']

        # Make stuff easier to parse
        row = {}
        row['gameId'] = game_id
        row['gameType'] = game_type
        row['homeTeam'] = data['homeTeam']['abbrev']
        row['awayTeam'] = data['awayTeam']['abbrev']
        row['homeScore'] = data['homeTeam']['score']
        row['awayScore'] = data['awayTeam']['score']

        for item in team_summary:
            category = item["category"]
            row[f"home{category[0].upper()}{category[1:]}"] = item["homeValue"]
            row[f"away{category[0].upper()}{category[1:]}"] = item["awayValue"]

        games.append(row)

game_stats = pl.DataFrame(games, strict=False)

